In [2]:
import pandas as pd

df = pd.read_csv('D:/p/Iowa_Liquor_Sales.csv')

C:\Users\user\AppData\Local\Temp\ipykernel_32116\1865551121.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:/p/Iowa_Liquor_Sales.csv')


In [3]:
# 使用特定的日期格式解析日期列
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# 筛选 2012 年到 2014 年之间的日期数据
start_date = '2012-01-01'
end_date = '2014-12-31'

# 筛选数据框
f_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

In [4]:
selected_columns = ['Item Description', 'Bottles Sold']
df_selected = f_df[selected_columns]
print(df_selected)

                       Item Description  Bottles Sold
100                             Absente             4
102                    Cruzan Mango Rum             3
103           Arrow Peppermint Schnapps             3
104           Uv Blue (raspberry) Vodka            12
106                       Kinky Liqueur            12
...                                 ...           ...
8197498             Cedar Ridge Bourbon             6
8197500                   Tanqueray Gin             2
8197501  Arrow Peppermint Schnapps Mini             1
8197502                Cabo Wabo Blanco             6
8197504   Jack Daniels Old #7 Black Lbl            12

[6243618 rows x 2 columns]


In [5]:
from scipy import stats
grouped_df = df_selected.groupby('Item Description')['Bottles Sold'].sum().reset_index()
sorted_df = grouped_df.sort_values(by='Bottles Sold', ascending=False).reset_index(drop=True)
# print(sorted_df)
# sorted_df.to_csv('D:/p/Iowa_Liquor_Sales_Sorted.csv', index=False)
combined_df = pd.concat([sorted_df.head(10), sorted_df.tail(10)], ignore_index=True)
# combined_df.to_csv('D:/p/Iowa_Liquor_Sales_f.csv', index=False)
# print(combined_df)
# print(sorted_df)
combined_df_t = sorted_df.head(912)


In [6]:
from scipy import stats
import numpy as np

# Calculate Z-scores for 'Bottles Sold'
combined_df_t['Z-score'] = np.abs(stats.zscore(combined_df_t['Bottles Sold']))

# Identify outliers (Z-score > 3)
outliers = combined_df_t[combined_df_t['Z-score'] > 3]

# Print or display outliers
print(outliers)

# Optionally, remove outliers
cleaned_df = combined_df_t[combined_df_t['Z-score'] <= 3]


                   Item Description  Bottles Sold    Z-score
0                      Black Velvet       3116129  15.625319
1                     Hawkeye Vodka       2407269  11.995246
2         Captain Morgan Spiced Rum       1630743   8.018656
3                Five O'clock Vodka       1602418   7.873604
4     Jack Daniels Old #7 Black Lbl       1144380   5.527991
5         Fireball Cinnamon Whiskey        928748   4.423740
6               Mccormick Vodka Pet        873886   4.142791
7             Smirnoff Vodka 80 Prf        871548   4.130818
8                      Barton Vodka        858680   4.064921
9       Crown Royal Canadian Whisky        794523   3.736373
10             Bacardi Superior Rum        785503   3.690182
11                   Phillips Vodka        778615   3.654909
12             Jagermeister Liqueur        760497   3.562126
13     Absolut Swedish Vodka 80 Prf        760446   3.561865
14      Seagrams 7 Crown Bl Whiskey        741778   3.466266
15  Paul Masson Grande A

C:\Users\user\AppData\Local\Temp\ipykernel_32116\1390572098.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_t['Z-score'] = np.abs(stats.zscore(combined_df_t['Bottles Sold']))


In [12]:
import os
import warnings
warnings.filterwarnings('ignore')  # Ignore all warnings

# 定义图片文件夹路径
image_folder = 'D:\p\liquor_images'

# 生成图片文件名列表
image_filenames = [f'l{i}.png' for i in range(18, 913)]

# 创建完整的图片路径列表
image_paths = [os.path.join(image_folder, filename) for filename in image_filenames]

# 将图片路径添加到 cleaned_df 数据框中
cleaned_df['image_paths'] = image_paths
cleaned_df.to_csv('D:/p/Iowa_Liquor_Sales_f.csv', index=False)

<>:6: SyntaxWarning: invalid escape sequence '\p'
<>:6: SyntaxWarning: invalid escape sequence '\p'
C:\Users\user\AppData\Local\Temp\ipykernel_32116\961838997.py:6: SyntaxWarning: invalid escape sequence '\p'
  image_folder = 'D:\p\liquor_images'


In [13]:
import numpy as np
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import sys
import contextlib

# Suppresses TensorFlow info and warnings, shows only errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Load pre-trained VGG16 model (without the top classification layer)
base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

# Function to suppress stdout (to hide the output during feature extraction)
@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            
def extract_features(img_path):
    if os.path.exists(img_path):
        # Load image and resize to (224, 224) as required by VGG16
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)  # Preprocess the image as expected by VGG16
        
        # Extract features using VGG16 model without progress bar (verbose=0)
        features = model.predict(img_data, verbose=0)
        return features.flatten()  # Flatten the features for further use
    else:
        print(f"File not found: {img_path}")
        return None
cleaned_df['image_paths'] = cleaned_df['image_paths'].str.replace('.png', '.jpg')
# Apply feature extraction to your dataframe (assuming 'cleaned_df' has a column 'image_paths')
cleaned_df['features'] = cleaned_df['image_paths'].apply(extract_features)

# Optional: check extracted features
# print(cleaned_df.head())

In [14]:
# 将特征和销售量数据提取成新的数据集
features = np.array(cleaned_df['features'].tolist())
sales = cleaned_df['Bottles Sold'].values
sales_log = np.log1p(sales)
# 检查特征和销售量的形状
print(features.shape)
print(sales_log.shape)

(895, 512)
(895,)


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
# Scale the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, sales_log, test_size=0.2, random_state=42)

model = Sequential([
    Dense(128, input_dim=features_scaled.shape[1], activation='relu', kernel_regularizer=l2(0.001)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=1000, batch_size=16, validation_split=0.2, verbose=0)

# Evaluate the model
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Print a few predictions
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred.flatten()  # Flatten to match dimensions
})
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppresses TensorFlow info and warnings, shows only errors
print(results_df.head(5))


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step  
Mean Squared Error: 3.483029296866744
      Actual  Predicted
0   8.718827   8.806948
1   9.519148  10.642384
2   9.268137  10.409067
3   8.689633   7.538166
4  12.321858   9.524442


In [18]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Calculate MSE, MAE, and R-squared
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')


Mean Squared Error (MSE): 3.483029296866744
Mean Absolute Error (MAE): 1.480341076684357
